# GCC's ssc-cdi : ptychography pipeline for EMA v0.7.12
 
- #### Questions? E-mail gcc@lnls.br
- #### Documentation about the ssc-cdi package: https://gcc.lnls.br/wiki/

## Imports

In [1]:
import os, json, h5py
import numpy as np
import scipy.ndimage as scp
import matplotlib.pyplot as plt
from matplotlib.colors import LogNorm
import sscInstall
import sscCdi
import sscResolution
import time
from PIL import Image

print(f'sscCdi version: {sscCdi.__version__}')
print(f'sscResolution version: {sscResolution.__version__}')

sscCdi version: 0.6.40
sscResolution version: 0.6


## Inputs

In [2]:
input_dict = {
    
"beamline": "EMA",

"data_path": "/ibira/lnls/beamlines/ema/proposals/20210094/data/ptycho-files/ptycho_file_001.h5",
"beamline_parameters_path": "/ibira/lnls/beamlines/ema/proposals/20210094/data/ptycho-files/ptycho_aux.h5",
"output_path": "/ibira/lnls/beamlines/ema/proposals/20210094/proc/ptycho-files", # you must create the proc folder before run, or give a valid output path
 
    
"positions_unit_conversion": 1.0,

"fresnel_regime": False,
    
"flatfield": "/ibira/lnls/labs/ldet/mobipix_15D/mobipix_007/01_Calibracao/Flat Field/100imgs/Ge_FF.tif", # path to flat fiedl
"mask": "", # path to mask, must be in .tif

"CPUs": 32,
"GPUs": [0],

'binning':1,
    
"detector_distance": 2.9, # meters
    
"DP_center": [362,376], # [center_y, center_x]
"DP_radius": 100,  # crop radius

"position_rotation": 0.0,

"object_padding": 30,

"incoherent_modes": 1,
"probe_support": {"type": "cross",  "center_width": 90,  "cross_width": 70, "border_padding": 10}, 

"initial_obj": {"obj": 'random'}, 
"initial_probe": { "probe": 'inverse'},
    
'algorithms': { 

    '1': {'name':'RAAR',
         'iterations': 100, 
         'beta': 0.795,
         'step_object': 1.0,
         'step_probe': 1.0,   
         'regularization_object': 0.01,
         'regularization_probe': 0.01,
         'momentum_obj': 0.0,
         'momentum_probe': 0.0, 
         'batch': 64,
    },  

    '2': {'name':'AP',
         'iterations': 100, 
         'step_object': 1.0,
         'step_probe': 1.0,   
         'regularization_object': 0.01,
         'regularization_probe': 0.01,
         'momentum_obj': 0.4,
         'momentum_probe': 0.4, 
         'batch': 64,
    },
    
    '3': {'name':'RAAR',
         'iterations': 100, 
         'beta': 0.795,
         'step_object': 1.0,
         'step_probe': 1.0,   
         'regularization_object': 0.01,
         'regularization_probe': 0.01,
         'momentum_obj': 0.0,
         'momentum_probe': 0.0, 
         'batch': 64,
         'initial_obj': {"obj": 'random'},
    }, 
    
    '4': {'name':'AP',
         'iterations': 100, 
         'step_object': 1.0,
         'step_probe': 1.0,   
         'regularization_object': 0.01,
         'regularization_probe': 0.01,
         'momentum_obj': 0.4,
         'momentum_probe': 0.4, 
         'batch': 64,
    },
    
    '5': {'name':'RAAR',
         'iterations': 100, 
         'beta': 0.795,
         'step_object': 1.0,
         'step_probe': 1.0,   
         'regularization_object': 0.01,
         'regularization_probe': 0.01,
         'momentum_obj': 0.0,
         'momentum_probe': 0.0, 
         'batch': 64,
         # 'initial_obj': {"obj": 'random'},
    }, 
    
    '6': {'name':'AP',
         'iterations': 100, 
         'step_object': 1.0,
         'step_probe': 1.0,   
         'regularization_object': 0.01,
         'regularization_probe': 0.01,
         'momentum_obj': 0.4,
         'momentum_probe': 0.4, 
         'batch': 64,
    },

    }
}

input_dict = sscCdi.ema.ema_ptycho_processing.define_paths(input_dict)

	Data path:  /ibira/lnls/labs/tepui/proposals/20210062/camila/ema/proposals/00000000/data/ptycho-files/ptycho_file_001.h5
	Output path: /ibira/lnls/labs/tepui/proposals/20210062/camila/ema/proposals/00000000/proc/ptycho-files


## Cropping diffraction patterns

In [ ]:
# getting diffraction patterns
data = h5py.File(input_dict["data_path"],'r')
diffraction_patterns = data["entry/data/data"][()]
diffraction_patterns = diffraction_patterns.astype(np.float32)

# appling mask and flatfield
flat_field = np.array(Image.open(input_dict["flatfield"])).astype(np.float32)
mask = np.array(Image.open(input_dict["mask"])).astype(np.float32)

for i in range(diffraction_patterns.shape[0]):
    diffraction_patterns[i,:,:] = diffraction_patterns[i,:,:]*flat_field
    diffraction_patterns[i,:,:][mask==1] = -1 # setting bad pixels as -1

# cropping data
diffraction_patterns_cropped = sscCdi.ema.crop_data(input_dict, diffraction_patterns)

# aditional masking
# diffraction_patterns_cropped[diffraction_patterns_cropped==0] = -1
diffraction_patterns_cropped[:,94:107,95:104] = -1

# visualization
from matplotlib.colors import LogNorm
fig, ax = plt.subplots(1,1,dpi=150,figsize=(4,4))
ax.imshow(diffraction_patterns_cropped[0],norm=LogNorm());

## Ptychography

In [3]:
# calling ptycho
obj, probe, input_dict = sscCdi.ema.ema_ptycho_processing.ema_ptychography(input_dict,diffraction_patterns_cropped)


	Object pixel size = 0.16 nm
	Limit thickness for resolution of 1 pixel: 0.004 microns
Fresnel number: -0.001
Creating initial probe...
	Probe shape (512,512) with 1 incoherent mode(s)
Setting probe support...
Creating initial object...
Diffraction Patterns: (8, 512, 512)
Initial Object: (12914, 6264)
Initial Probe: (1, 512, 512)
Probe Support: (1, 512, 512)
Probe Positions: (8, 4)
Creating datapack...
Total datapack size: 0.66 GBs
Starting ptychography... using 1 GPUs [0] and 32 CPUs
	Calling 70 iterations of RAAR algorithm...
	Done in: 2.56 seconds
	Calling 50 iterations of positioncorrection algorithm...
	Done in: 1.82 seconds
	Calling 5 iterations of GL algorithm...
	Done in: 0.29 seconds


In [4]:
fig, ax = plt.subplots(1,2,dpi=150,figsize=(6,3))
ax[0].imshow(np.abs(probe[0,0]),cmap='gray'); # [cy1:cy2,cx1:cx2]
ax[1].imshow(np.angle(obj[0]),cmap='gray');  

## Unwrap

In [ ]:
import sscCdi
from skimage.restoration import unwrap_phase

recon_obj_phase_unwrap = unwrap_phase(np.angle(obj))
fig, ax = plt.subplots(1,2,dpi=150,figsize=(10,4))
ax[0].imshow(np.abs(obj),cmap='gray'); # [cy1:cy2,cx1:cx2]
ax[1].imshow(recon_obj_phase_unwrap,cmap='gray');

## Fourier Ring Correlation

In [ ]:
ny_obj = obj.shape[1]
nx_obj = obj.shape[2]

if ny_obj < nx_obj:
    new_obj = obj[0,:,0:ny_obj]
elif nx_obj < ny_obj:
    new_obj = obj[0,0:nx_obj,:]
else:
    new_obj = obj[0]
    
print(new_obj.shape)

In [ ]:
img = np.angle(new_obj)
# img = img - img.min()

padding = 0
sharpness = 5
radius = 0.5

new_img = sscResolution.frc.window( img, padding, [sharpness, radius] )
dic = sscResolution.frc.computep( new_img , 1 ) 
print(f"\tResolution via halfbit criterion: {1e9*input_dict['object_pixel']/dic['x']['H']:.2f} nm")

sscResolution.frc.plot(dic, {'label': "Resolution", 'unit': "nm", 'pxlsize': input_dict['object_pixel']*1e9} )